In [ ]:
import progressbar
import h5py
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pytz
import warnings
import pylab as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import search_dispersion_events 
import importlib
importlib.reload(search_dispersion_events)

In [ ]:
df = pd.read_csv('data/train.csv', parse_dates=['start_time', 'end_time'])
df.head()

In [ ]:
def visualize(row, plot=True):
    importlib.reload(search_dispersion_events)
    
    # Do computation --------------------------------------------------
    fh = search_dispersion_events.read_file(row.filename)

    dEicdt_smooth, Eic = search_dispersion_events.estimate_log_Eic_smooth_derivative(fh)

    df_match, integrand,integral, upper_area_frac = search_dispersion_events.walk_and_integrate(
        fh, dEicdt_smooth, Eic, search_dispersion_events.DEFAULT_INTERVAL_LENGTH,
        return_integrand=True
    )
    
    if not plot:
        return df_match
    
    # Do plotting --------------------------------------------------
    i = fh['t'].searchsorted(row.start_time)
    j = fh['t'].searchsorted(row.end_time)
    
    fig, axes = plt.subplots(5, 1, figsize=(18, 16), sharex=True)

    im = axes[0].pcolor(fh['t'][i:j], np.log10(fh['ch_energy']), fh['ion_d_ener'][:, i:j], 
                        norm=LogNorm(vmin=1e3, vmax=1e8), cmap='jet')
    plt.colorbar(im, ax=axes[0]).set_label('Log Energy Flux')
    plt.colorbar(im, ax=axes[1]).set_label('')
    plt.colorbar(im, ax=axes[2]).set_label('')
    plt.colorbar(im, ax=axes[3]).set_label('')
    plt.colorbar(im, ax=axes[4]).set_label('')

    axes[0].plot(fh['t'][i:j], Eic[i:j], 'b*-')
    axes[0].invert_yaxis()
    axes[0].set_ylabel('Log Energy [eV] - Ions')

    time_length = row.end_time - row.start_time
    fig.suptitle(f'{time_length.total_seconds() / 60:.1f} minutes : '
                 f'{row.start_time.isoformat()} - {row.end_time.isoformat()}', fontweight='bold')

    title = 'MLAT = (%.1f deg to %.1f deg)' % (fh['mlat'][i], fh['mlat'][j])
    title += ' Northward' if fh['mlat'][j] > fh['mlat'][i] else ' Southward'
    title += f' -- Integral {float(integral[i:j].max()):.2f} -- UpperFrac {float(upper_area_frac[i:j].max()):.2f}'
    axes[0].set_title(title)

    axes[1].fill_between(fh['t'][i:j], 0, integrand[i:j])
    axes[1].axhline(0, color='black', linestyle='dashed')
    axes[1].set_ylim([-.25, .25])
    axes[1].set_ylabel('D(t) [eV/s]')

    axes[2].plot(fh['t'][i:j], np.log10(fh['density'][i:j]+.01), color='black')
    axes[2].set_ylabel('Log Density')
    axes[2].axhline(search_dispersion_events.DEFAULT_DENSITY_LOG_THRESHOLD, color='red', linestyle='dashed')
    axes[2].set_ylim([-2, 10])
    
    axes[3].plot(fh['t'][i:j], fh['peak_flux'][i:j])
    axes[3].set_ylabel('Peak Flux')
    axes[3].set_yscale('log')
#    axes[3].axhline(search_dispersion_events.DEFAULT_DENSITY_LOG_THRESHOLD, color='red', linestyle='dashed')
#    axes[3].set_ylim([-2, 10])

        
    en_inds = np.log10(fh['ch_energy']).searchsorted(Eic[i:j])
    en_inds = [min(i, 18) for i in en_inds]
    flux_at_Eic = fh['ion_d_ener'][en_inds, np.arange(i,j)]
    flux_at_Eic[np.isnan(Eic[i:j])] = np.nan
    
    axes[4].plot(fh['t'][i:j], flux_at_Eic, color='purple')
    axes[4].set_ylabel('Flux at Eic')
    axes[4].set_yscale('log')
    axes[4].axhline(10**6, color='red', linestyle='dashed')
    
    return df_match

In [ ]:
visualize(df[df['class']==1].iloc[0])

In [ ]:
for i in [50, 79, 123, 146, 165, 167, 194, 200, 201, 203, 218, 220, 225, 228, 238, 246, 266, 295, 301, 304]:
    visualize(df.iloc[i])